In [30]:
jb = JusterB.new_with_deposit('A', 100, 300)
insurance = jb.insure('B', 200, 'for')

removed_value = 400
jb.pools * (jb.balances['contract'] - removed_value) / removed_value

<Pools>
{
    "for": 75.0,
    "against": 25.0
}

### А что если пулы всегда держать нормализованными и держать просто на контракте текущую ликвидность? вообще отойти от имеющихся идей

In [33]:
jb.pools / jb.pools.min()

<Pools>
{
    "for": 3.0,
    "against": 1.0
}

### new test:

In [1]:
from importlib import reload
import deposit, agreement, juster_b, lock
reload(lock); reload(deposit); reload(agreement); reload(juster_b)
from juster_b import JusterB
from pools import Pools

# the case when one wins with one pool and other with another:

jb = JusterB.new_with_deposit('A', 100, 100)
insurance_one = jb.insure('B', 9900, 'against')

assert jb.pools == Pools(1, 10000)

In [2]:
jb

<Line>
{
    "pools": {
        "for": 1.0,
        "against": 10000
    },
    "total_shares": 100,
    "is_claimed": false,
    "agreements": {
        "0": {
            "user": "B",
            "pool": "against",
            "amount": 9900,
            "delta": 99.0
        }
    },
    "deposits": {
        "A": {
            "amount": 100,
            "shares": 100,
            "pools": {
                "for": 100,
                "against": 100
            }
        }
    },
    "locks": {},
    "balances": {
        "A": -100,
        "contract": 10000,
        "B": -9900
    }
}

In [3]:
jb.give_reward(insurance_one)

In [4]:
jb

<Line>
{
    "pools": {
        "for": 9.999999999998899e-05,
        "against": 0.9999999999998899
    },
    "total_shares": 100,
    "is_claimed": false,
    "agreements": {},
    "deposits": {
        "A": {
            "amount": 100,
            "shares": 100,
            "pools": {
                "for": 100,
                "against": 100
            }
        }
    },
    "locks": {},
    "balances": {
        "A": -100,
        "contract": 1.0,
        "B": 99.0
    }
}

In [5]:
jb.inflation

10000.0000000011

In [6]:
delta = 99
amount = 9900
max_pools = 10000

left_liquidity = max_pools - amount - delta

# (1 + (max_pools - amount))*delta
amount / max_pools
max_pools / (max_pools - amount)

100.0

In [7]:
1 - (amount + delta) / max_pools

9.999999999998899e-05

In [8]:
1 / (1 - (amount + delta) / max_pools)

10000.0000000011

In [9]:
1 / (1 - (amount) / max_pools)

99.99999999999991

In [10]:
jb

<Line>
{
    "pools": {
        "for": 9.999999999998899e-05,
        "against": 0.9999999999998899
    },
    "total_shares": 100,
    "is_claimed": false,
    "agreements": {},
    "deposits": {
        "A": {
            "amount": 100,
            "shares": 100,
            "pools": {
                "for": 100,
                "against": 100
            }
        }
    },
    "locks": {},
    "balances": {
        "A": -100,
        "contract": 1.0,
        "B": 99.0
    }
}

In [11]:
# Pools should be reduced here, but there are no such kind of logic in model yet:
# assert jb.pools == Pools(0.0001, 1)

insurance_two = jb.insure('C', 1, 'for')
# assert jb.pools == Pools(1, 0.0001)

In [12]:

jb.claim_insurance_case()

jb.give_reward(insurance_two)

In [13]:
lock = jb.lock_liquidity('A', 100)
jb.withdraw_lock(lock)
# jb.assert_empty()

jb

InvalidState: Negative amount on contract

In [16]:
from importlib import reload
import deposit, agreement, juster_b, lock
reload(lock); reload(deposit); reload(agreement); reload(juster_b)
from juster_b import JusterB

jb = JusterB.new_with_deposit('A', 1000, 1000)
insurance_one = jb.insure('B', 1000, 'for')
lock = jb.lock_liquidity('A', 500)
insurance_two = jb.insure('C', 1000, 'against')
jb.claim_insurance_case()
jb.give_reward(insurance_one)
jb.withdraw_lock(lock)
jb.give_reward(insurance_two)
lock = jb.lock_liquidity('A', 500)
jb.withdraw_lock(lock)

# jb.assert_empty()

# - test case with 50% liquidity locked, then claim insurance and remove liquidity (should be the same result)
# - test case where 50% liquidity locked, then some bets, then remove liquidity, then claim insurance, then remove another 50%
# - test case where claim_insurance_case called after lock created and after it should be freed (provider should receive profit from against pool)


# - test case when someone put a lot of liquidity in the end and get it back
# - test non symetrical event start
# - mix multiple cases

# start to check that there are no arbitrague opportunities


# что случится если до лока ликвидности были ставки, страховой случай не реализовался,
# пользователи виздровнули эти ставки, потом хоп страховой случай и провайдер тогда по какому сценарию выводит лок?
# -- надо в локе указывать время завершения лока и если страховой случай после него - значит по сценарию до лока

# нужно анализировать ещё момент наступления страхового случая потому что если пользователь не виздровнул
# и дождался страхового случая уже за пределами своего соглашения, этот случай на него не должен распространяться

# 2. также если после лока были ставки, они по идее вообще не должны влиять на результат провайдера


# 3. можно в модельку добавить каунтер, чтобы каждая операция связывалась со своим временем абстрактным, каждая следующая
# операция это новое время
# 4. можно добавить ещё .wait(time)

# подумать нет ли смысла объединить withdraw_lock & give_reward?

jb

<Line>
{
    "actual_pools": {
        "for": 0.0,
        "against": 0.0
    },
    "total_shares": 0,
    "is_claimed": true,
    "agreements": {},
    "deposits": {
        "A": {
            "amount": 0.0,
            "shares": 0.0,
            "pools": {
                "for": 0.0,
                "against": 0.0
            }
        }
    },
    "locks": {},
    "balances": {
        "A": 500.0,
        "B": 500.0,
        "C": -1000
    }
}

In [17]:
# THIS CASE AGAIN:
import pdb
from pools import Pools

# the case with insurance case while partial liquidity lock:
jb = JusterB.new_with_deposit('A', 1000, 1000)
insurance_one = jb.insure('B', 1000, 'for')
assert jb.pools == Pools(2000, 500)

lock = jb.lock_liquidity('A', 500)
assert jb.locked_pools == Pools(1000, 250)

# C possible win is 800 = 1000 / 1250 * 1000:
insurance_two = jb.insure('C', 1000, 'against')
actual_pools = jb.pools - jb.locked_pools
assert actual_pools == Pools(200, 1250)
assert jb.pools == Pools(1200, 1500)

jb.claim_insurance_case()
jb.give_reward(insurance_one)
# at this moment:
# - B wins 500 and need to return his 1000
# - A loses 500 and it is splitted between actual pool (-250) and locked_pool (-250)

jb.give_reward(insurance_two)

# TODO replace two next and should not have any effect
# jb.withdraw_lock(lock) 
# lock = jb.lock_liquidity('A', 500)
# jb.withdraw_lock(lock)
jb

import pdb; pdb.run('jb.withdraw_lock(lock)')

# ?? Вывод провайдера не должен зависеть от результата, так как в одном случае он проиграл с for, в другом выйграл с against
# хотя нет, вроде как сходится что результат то один
# pdb.run('jb.give_reward(insurance_one)')
jb


# МОЖЕТ БЫТЬ:
# Участник при выходе не должен уже никак влиять на соотношение пулов, но он может их раздувать сдувать увеличивая
# минимальную часть пула на забираемую / освобождаемую ликвидность

AssertionError: 

In [18]:
# the case when provider first wins with case "for" and then loses with case "for"

---
а может пулы должны быть отвязаны от общей ликвидности? то есть должна быть переменная которая показывает сколько общей ликвидности в контракте и пулы которые никак с ней не связаны (но что-то мне в этом не нравится)

---
а не по минимальному ли пулу надо сокращать ликвидность и шринкать пулы?

In [19]:
from importlib import reload
import deposit, agreement, juster_b, lock
reload(lock); reload(deposit); reload(agreement); reload(juster_b)
from juster_b import JusterB

jb = JusterB.new_with_deposit('A', 1000, 1000)
insurance_one = jb.insure('B', 1000, 'for')
lock = jb.lock_liquidity('A', 1000)
jb.claim_insurance_case()
jb.give_reward(insurance_one)
jb.withdraw_lock(lock)
jb.assert_empty()
jb

# import pdb
# pdb.run('jb.withdraw_lock(lock)')

<Line>
{
    "actual_pools": {
        "for": 0.0,
        "against": 0.0
    },
    "total_shares": 0,
    "is_claimed": true,
    "agreements": {},
    "deposits": {
        "A": {
            "amount": 0.0,
            "shares": 0.0,
            "pools": {
                "for": 0.0,
                "against": 0.0
            }
        }
    },
    "locks": {},
    "balances": {
        "A": -500.0,
        "B": 500.0
    }
}

## Идея как-то исключать залоченную ликвидность из пулов, но не определять пока победила она или нет, возможно ли это?
- то есть для новых ставок уже должно считаться что ликвидности меньше и пулы другие
- а при виздро провайдером чтобы были учтены только те ставки которые были на момент его выхода


- вообще ведь провайдер при выходе одинаково влияет на пулы всегда и просто шринкает их пропорционально выходу
- но на размер того что он в итоге получает влияет размер пулов в ивенте. Вот вопрос: нужно ли сохранять может быть эти пулы на момент запроса выхода? и потом рассчитывать прибыль / убыток по этим пулам? Потому что явно ведь не по новым пулам?
    - или всё-таки даже пулы не надо записывать будет?

- хочу понять: влияет ли результат в зависимости от того кто первый выходит пользователь или провайдер?